<font size="5"> Downloading all dependancies that are needed</font>

In [91]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library
import requests # library to handle requests
from pandas.io.json import json_normalize 
# import k-means from clustering stage
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
#https://cocl.us/new_york_dataset   -data set to be used

In [92]:
import json # library to handle JSON files

<font size="4"> Download and explore dataset</font>

In [93]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [94]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

NewYork_data=newyork_data['features']

In [95]:
new_york_data = newyork_data['features']

<font size="4"> Transforming data into a pandas dataframe</font>

In [96]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
new_york = pd.DataFrame(columns=column_names)

In [97]:
for data in new_york_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    new_york = new_york.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [172]:
new_york.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


<font size="4"><h1>Let's first find the Borough with the greatest population. This will be used for further analysis in the project<h1></font>
    <font size="4"> Let's import the data set with population information and load it into a pandas dataframe</font>

In [173]:
pop_data=pd.read_csv("https://data.beta.nyc/dataset/0ff93d2d-90ba-457c-9f7e-39e47bf2ac5f/resource/7caac650-d082-4aea-9f9b-3681d568e8a5/download/nyc_zip_borough_neighborhoods_pop.csv"
)
pop_data.head()

,zip,borough,post_office,neighborhood,population,density
0,10001,Manhattan,"New York, NY",Chelsea and Clinton,21102,33959
1,10002,Manhattan,"New York, NY",Lower East Side,81410,92573
2,10003,Manhattan,"New York, NY",Lower East Side,56024,97188
3,10004,Manhattan,"New York, NY",Lower Manhattan,3089,5519
4,10005,Manhattan,"New York, NY",Lower Manhattan,7135,97048


 <font size="4"> Use pandas method idmax() to get the maximum values</font>

In [174]:
pop_data.loc[pop_data['population'].idxmax()]

zip                   11368
borough              Queens
post_office      Corona, NY
neighborhood    West Queens
population           109931
density               41768
Name: 139, dtype: object

 <font size="4"> Since Queens has the highest population, it will be used for further analysis </font>

In [175]:
#extracting Queens data only
queens_data = new_york[new_york['Borough'] == 'Queens'].reset_index(drop=True)
queens_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138


 <font size="4"> The next stage is to use geopy library to get coordinates for Queens. </font>

In [176]:
address = 'Queens, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Queens are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Queens are 40.7498243, -73.7976337.


 <font size="4"><h1> Using Foursquare to get venue and category information</h1></font>
     <font size="3">  Foursquare is a social location service that allows users to explore the world around them. The Foursquare API will be used to get the venue and category information..</font>

In [177]:
#Foursquare Credentials and Version
client_id = '21MWAQ0OAOF2KFBZ5DS1Y15C3HMEPLOQSKIMXKURGJA1TGNE' # your Foursquare ID
client_secret = 'AULST3BFPQUXEIZ2DSHGFDSVCE1X2QDTVWARCY4VZBOFY0R3' # your Foursquare Secret
VERSION = '20190530' # Foursquare API version

In [178]:
#Exploring first neighborhood
queens_data.loc[0, 'Neighborhood']

'Astoria'

<font size="3">Now will retrieve the 100 most popular spots for each neighborhood,within 500m radius </font>

In [179]:
radius = 500
limit = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(client_id, client_secret, latitude, longitude, VERSION, radius, limit)

In [106]:
res = requests.get(url).json()
res

{'meta': {'code': 200, 'requestId': '5fef03574536486d2ada76a7'},
 'response': {'headerLocation': 'Flushing',
  'headerFullLocation': 'Flushing, Queens',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 40.7543243045, 'lng': -73.79170471608292},
   'sw': {'lat': 40.7453242955, 'lng': -73.8035626839171}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4dd9a5cfc65bee535ae8e26f',
       'name': 'Pado Sushi Restaurant',
       'location': {'address': '171-53 46th Ave',
        'crossStreet': 'Utopia Pkwy.',
        'lat': 40.75359944231735,
        'lng': -73.79442522679902,
        'labeledLatLngs': [{'label': 'display',
          'lat': 40.75359944231735,
          'lng': -73.79442522679902}],
        'distance': 499,


In [180]:
#Function to get category of venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [181]:
import json 
from pandas.io.json import json_normalize

In [182]:
venues = res['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-182-e7f95331d398>:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Pado Sushi Restaurant,Korean Restaurant,40.753599,-73.794425
1,Flushing Marina,Boat or Ferry,40.749661,-73.799772
2,Western Riding Club,Sports Club,40.749359,-73.797699
3,Underhill Ave Bike Trail,Bike Trail,40.747286,-73.799391
4,MTA - Q65 Bus Stop,Bus Station,40.748537,-73.801613


In [183]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


<font size="4"> Let's explore neighborhoods in Queens</font>

In [184]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            client_id, 
            client_secret, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [112]:
queens_venues = getNearbyVenues(names=queens_data['Neighborhood'],
                                   latitudes=queens_data['Latitude'],
                                   longitudes=queens_data['Longitude']
                                  )

In [113]:
print(queens_venues.shape)
queens_venues.head()

(2088, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Astoria,40.768509,-73.915654,Favela Grill,40.767348,-73.917897,Brazilian Restaurant
1,Astoria,40.768509,-73.915654,Orange Blossom,40.769856,-73.917012,Gourmet Shop
2,Astoria,40.768509,-73.915654,Titan Foods Inc.,40.769198,-73.919253,Gourmet Shop
3,Astoria,40.768509,-73.915654,CrossFit Queens,40.769404,-73.918977,Gym
4,Astoria,40.768509,-73.915654,Simply Fit Astoria,40.769114,-73.912403,Gym


In [185]:
#Venues returned by each neighborhood
queens_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Arverne,20,20,20,20,20,20
Astoria,99,99,99,99,99,99
Astoria Heights,13,13,13,13,13,13
Auburndale,20,20,20,20,20,20
Bay Terrace,36,36,36,36,36,36
Bayside,73,73,73,73,73,73
Bayswater,2,2,2,2,2,2
Beechhurst,16,16,16,16,16,16
Bellaire,13,13,13,13,13,13


In [186]:
#Unique categories
print('There are {} uniques categories.'.format(len(queens_venues['Venue Category'].unique())))

There are 270 uniques categories.


<font size="5"><h1>Analyze each neighborhood</h1></font>

In [187]:
# one hot encoding
queens_onehot = pd.get_dummies(queens_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
queens_onehot['Neighborhood'] = queens_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [queens_onehot.columns[-1]] + list(queens_onehot.columns[:-1])
queens_onehot = queens_onehot[fixed_columns]

queens_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport Terminal,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Beach,Beach Bar,Beer Garden,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Cafeteria,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool Hall,Post Office,Pub,Ramen Restaurant,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train,Train Station,Turkish Restaurant,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

<font size="3">Next, let's group rows by neighborhood and by taking the mean of the frequency for each category</font>

In [188]:
newqueens_grouped = queens_onehot.groupby('Neighborhood').mean().reset_index()
newqueens_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport Terminal,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Beach,Beach Bar,Beer Garden,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Cafeteria,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool Hall,Post Office,Pub,Ramen Restaurant,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train,Train Station,Turkish Restaurant,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Arverne,0.000000,0.000000,0.000000,0.000000,0

<font size="4">Let's find the 10 most common  venues </font>

In [189]:
#Find 10 most common venues in each neighborhood



def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = newqueens_grouped['Neighborhood']

for ind in np.arange(newqueens_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(newqueens_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arverne,Surf Spot,Metro Station,Sandwich Place,Playground,Donut Shop,Burrito Place,Bus Stop,Café,Board Shop,Beach
1,Astoria,Bar,Middle Eastern Restaurant,Seafood Restaurant,Greek Restaurant,Hookah Bar,Mediterranean Restaurant,Café,Indian Restaurant,Bakery,Gym / Fitness Center
2,Astoria Heights,Plaza,Bakery,Motel,Bus Station,Bowling Alley,Supermarket,Chinese Restaurant,Liquor Store,Burger Joint,Hostel
3,Auburndale,Italian Restaurant,Hookah Bar,Train,Mattress Store,Fast Food Restaurant,Supermarket,Noodle House,Bar,Miscellaneous Shop,Toy / Game Store
4,Bay Terrace,Clothing Store,Women's Store,Cosmetics Shop,Donut Shop,Kids Store,American Restaurant,Shoe Store,Bank,Mobile Phone Shop,Steakhouse


<font size="3">Run k-means to cluster the neighborhood into 5 clusters.</font>

In [190]:
# set number of clusters
kclusters = 5

t_clustering = newqueens_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(t_clustering)
kmeans.labels_[0:10]


array([1, 1, 1, 1, 1, 1, 1, 1, 3, 3], dtype=int32)

In [191]:
#add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

queens_merged = queens_data

# merge  to add latitude/longitude for each neighborhood
queens_merged = queens_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

queens_merged.head() 

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Queens,Astoria,40.768509,-73.915654,1,Bar,Middle Eastern Restaurant,Seafood Restaurant,Greek Restaurant,Hookah Bar,Mediterranean Restaurant,Café,Indian Restaurant,Bakery,Gym / Fitness Center
1,Queens,Woodside,40.746349,-73.901842,1,Grocery Store,Latin American Restaurant,Filipino Restaurant,Bar,Bakery,Thai Restaurant,Donut Shop,American Restaurant,Pub,Sandwich Place
2,Queens,Jackson Heights,40.751981,-73.882821,1,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Bakery,Thai Restaurant,Mobile Phone Shop,Grocery Store,Empanada Restaurant,Mexican Restaurant,Shoe Store
3,Queens,Elmhurst,40.744049,-73.881656,1,Thai Restaurant,Mexican Restaurant,Pizza Place,Chinese Restaurant,Vietnamese Restaurant,Bubble Tea Shop,Bank,Colombian Restaurant,Donut Shop,Salon / Barbershop
4,Queens,Howard Beach,40.654225,-73.838138,1,Pharmacy,Italian Restaurant,Fast Food Restaurant,Sandwich Place,Shipping Store,Sushi Restaurant,Supermarket,Mexican Restaurant,Motel,Seafood Restaurant


<font size="3">Visualizing clusters in a map</font>

In [192]:
 #create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
      queens_merged['Latitude'], 
      queens_merged['Longitude'], 
       queens_merged['Neighborhood'], 
       queens_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<font size="4"> Now, let's examine each and every cluster to get more insight. The following will be examined in each cluster:</font>
<ol>
    <li>if Italian restaurant is the most common venue </li>
    <li>number of Italian Restaurants in that cluster </li>
    <li>Total Number of restaurants in the cluster </li>
</ol>


In [224]:
#getting list of all restaurants in queens
contain_values = queens_venues[queens_venues['Venue Category'].str.contains('Restaurant')]
contain_values

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Astoria,40.768509,-73.915654,Favela Grill,40.767348,-73.917897,Brazilian Restaurant
5,Astoria,40.768509,-73.915654,Off The Hook,40.767200,-73.918104,Seafood Restaurant
7,Astoria,40.768509,-73.915654,Al-sham Sweets and Pastries,40.768077,-73.911561,Middle Eastern Restaurant
9,Astoria,40.768509,-73.915654,The Gully,40.766719,-73.912410,Indian Restaurant
11,Astoria,40.768509,-73.915654,Zyara Restaurant,40.766591,-73.912713,Restaurant
12,Astoria,40.768509,-73.915654,Duzan,40.768730,-73.911013,Falafel Restaurant
13,Astoria,40.768509,-73.915654,AbuQir Seafood,40.768577,-73.911169,Seafood Restaurant
15,Astoria,40.768509,-73.915654,Sabry's,40.768446,-73.911267,Seafood Restaurant
20,Astoria,40.768509,-73.915654,Jujube Tree,40.765309,-73.918162,Vegetarian / Vegan Restaurant
23,Astoria,40.768509,-73.915654,"Farid Kebab, Steinway",40.767548,-73.911972,Mediterranean Restaurant


In [228]:
#group restaurants by neighborhood and count
rest_per_neigh = contain_values.groupby("Neighborhood")["Venue Category"].count()
rest_per_neigh

Neighborhood
Arverne                  1
Astoria                 37
Astoria Heights          2
Auburndale               4
Bay Terrace              3
Bayside                 27
Beechhurst               3
Bellaire                 3
Belle Harbor             2
Bellerose                6
Blissville               2
Briarwood                1
Cambria Heights          4
College Point            9
Corona                   8
Douglaston               4
Edgemere                 2
Elmhurst                20
Far Rockaway             6
Floral Park              2
Flushing                21
Forest Hills             6
Forest Hills Gardens     3
Fresh Meadows            3
Glen Oaks                3
Hammels                  1
Hillcrest                1
Hollis                   1
Howard Beach            11
Hunters Point           18
Jackson Heights         35
Jamaica Center           7
Jamaica Estates          1
Jamaica Hills           12
Kew Gardens              9
Kew Gardens Hills        4
Laurelton      

In [193]:
#Cluster 1
df=queens_merged.loc[queens_merged['Cluster Labels'] == 0, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

In [194]:
df

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,Breezy Point,Beach,Monument / Landmark,Trail,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant
50,Neponsit,Beach,Women's Store,Filipino Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food Truck


In [195]:
#Finding if Italian Restaurant is the 1st most common resturant in the cluster
df.loc[df['1st Most Common Venue']=='Italian Restaurant']

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [196]:
#Findind number of Italian Restaurants in a cluster

df[df == 'Italian Restaurant'].count()


Neighborhood              0
1st Most Common Venue     0
2nd Most Common Venue     0
3rd Most Common Venue     0
4th Most Common Venue     0
5th Most Common Venue     0
6th Most Common Venue     0
7th Most Common Venue     0
8th Most Common Venue     0
9th Most Common Venue     0
10th Most Common Venue    0
dtype: int64

<font size='3'>There is no Italian Restaurant in this cluster. There are only 6 restaurants in this cluster and neither of them is a 1st common venue. </font>

In [197]:
#Cluster 2
df1=queens_merged.loc[queens_merged['Cluster Labels'] == 1, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

In [198]:
df1

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Astoria,Bar,Middle Eastern Restaurant,Seafood Restaurant,Greek Restaurant,Hookah Bar,Mediterranean Restaurant,Café,Indian Restaurant,Bakery,Gym / Fitness Center
1,Woodside,Grocery Store,Latin American Restaurant,Filipino Restaurant,Bar,Bakery,Thai Restaurant,Donut Shop,American Restaurant,Pub,Sandwich Place
2,Jackson Heights,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Bakery,Thai Restaurant,Mobile Phone Shop,Grocery Store,Empanada Restaurant,Mexican Restaurant,Shoe Store
3,Elmhurst,Thai Restaurant,Mexican Restaurant,Pizza Place,Chinese Restaurant,Vietnamese Restaurant,Bubble Tea Shop,Bank,Colombian Restaurant,Donut Shop,Salon / Barbershop
4,Howard Beach,Pharmacy,Italian Restaurant,Fast Food Restaurant,Sandwich Place,Shipping Store,Sushi Restaurant,Supermarket,Mexican Restaurant,Motel,Seafood Restaurant
5,Corona,Mexican Restaurant,Supermarket,Park,Convenience Store,Restaurant,South American Restaurant,Empanada Restaurant,Chinese Restaurant,Sandwich Place,Bakery
6,Forest Hills,Gym / Fitness Center,Gym,Yoga Studio,Pizza Place,Convenience Store,Park,Thai Restaurant,Pharmacy,Video Game Store,Supplement Shop
7,Kew Gardens,Bus Stop,Chinese Restaurant,Cosmetics Shop,Bar,Bank,Pharmacy,Donut Shop,Pizza Place,Indian Restaurant,Café
8,Richmond Hill,Pizza Place,Lounge,Latin American Restaurant,Bank,Mexican Restaurant,Donut Shop,Spanish Restaurant,Deli / Bodega,Metro Station,Moving Target
9,Flushing,Hotpot Restaurant,Korean Restaurant,Bubble Tea Shop,Bakery,Chinese Restaurant,Construction & Landscaping,Gym / Fitness Center,Gym,Asian Restaurant,Karaoke Bar


In [239]:
#Finding if Italian Restaurant is the 1st most common venue in each cluster
df1.loc[df1["1st Most Common Venue"]=='Italian Restaurant']


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Auburndale,Italian Restaurant,Hookah Bar,Train,Mattress Store,Fast Food Restaurant,Supermarket,Noodle House,Bar,Miscellaneous Shop,Toy / Game Store


In [240]:
#Findind number of Italian Restaurants in a cluster

df1[df1 == 'Italian Restaurant'].count()

Neighborhood              0
1st Most Common Venue     1
2nd Most Common Venue     2
3rd Most Common Venue     1
4th Most Common Venue     1
5th Most Common Venue     1
6th Most Common Venue     0
7th Most Common Venue     0
8th Most Common Venue     0
9th Most Common Venue     0
10th Most Common Venue    0
dtype: int64

<font size='3'>There are 6 Italian Restaurant in this cluster. These 6 Italian Restaurants are in the first 5 Most Common Venue.  This cluster has the highest number of restaurants</font>

In [201]:
#Cluster 3
df2=queens_merged.loc[queens_merged['Cluster Labels'] == 2, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

In [202]:
df2

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,Brookville,Deli / Bodega,Women's Store,Filipino Restaurant,Empanada Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop


In [203]:
#Finding if Italian Restaurant is the 1st most common venue in the cluster
df2.loc[df2['1st Most Common Venue']=='Italian Restaurant']

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [204]:
#Findind number of Italian Restaurants in a cluster

df2[df2 == 'Italian Restaurant'].count()

Neighborhood              0
1st Most Common Venue     0
2nd Most Common Venue     0
3rd Most Common Venue     0
4th Most Common Venue     0
5th Most Common Venue     0
6th Most Common Venue     0
7th Most Common Venue     0
8th Most Common Venue     0
9th Most Common Venue     0
10th Most Common Venue    0
dtype: int64

<font size='3'>This cluster doesn't have much restaurants since it only has one neighborhood and neither is there an Italian restaurant in it.</font>

In [205]:
#Cluster 4
df3=queens_merged.loc[queens_merged['Cluster Labels'] == 3, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

In [206]:
df3

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Glendale,Dance Studio,Deli / Bodega,Brewery,Food & Drink Shop,Pizza Place,Arts & Crafts Store,Filipino Restaurant,Falafel Restaurant,Farm,Farmers Market
19,South Ozone Park,Park,Deli / Bodega,Bar,Sandwich Place,Food Truck,Fast Food Restaurant,Grocery Store,Donut Shop,Food,Hotel
21,Whitestone,Dance Studio,Deli / Bodega,Bubble Tea Shop,Candy Store,Women's Store,Filipino Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
30,Briarwood,Deli / Bodega,Playground,Arts & Crafts Store,Plaza,Convenience Store,Coffee Shop,Gym,Indian Restaurant,Home Service,Hobby Shop
36,St. Albans,Caribbean Restaurant,Dance Studio,Health Food Store,Fried Chicken Joint,Fast Food Restaurant,Café,Shopping Mall,Seafood Restaurant,Donut Shop,Chinese Restaurant
41,Far Rockaway,Deli / Bodega,Chinese Restaurant,Pizza Place,Bus Station,Supermarket,Market,Fried Chicken Joint,Sandwich Place,Bank,Metro Station
42,Broad Channel,Other Nightlife,Sporting Goods Shop,Deli / Bodega,Pizza Place,Dive Bar,Filipino Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market
47,Edgemere,Metro Station,Hotel,Seafood Restaurant,Scenic Lookout,Farm,Park,Asian Restaurant,Skate Park,Bus Station,Pizza Place
49,Rockaway Beach,Beach,Latin American Restaurant,Ice Cream Shop,BBQ Joint,Arepa Restaurant,Seafood Restaurant,Metro Station,Bus Station,Food Stand,Burrito Place
52,Floral Park,Indian Restaurant,Pizza Place,Basketball Court,Dosa Place,Grocery Store,Wine Shop,Flower Shop,Flea Market,Fish Market,Food


In [207]:
#Finding if Italian Restaurant is the 1st most common venue in the cluster
df3.loc[df3['1st Most Common Venue']=='Italian Restaurant']

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [208]:
#Findind number of Italian Restaurants in a cluster

df3[df3 == 'Italian Restaurant'].count()

Neighborhood              0
1st Most Common Venue     0
2nd Most Common Venue     0
3rd Most Common Venue     0
4th Most Common Venue     0
5th Most Common Venue     1
6th Most Common Venue     0
7th Most Common Venue     0
8th Most Common Venue     0
9th Most Common Venue     0
10th Most Common Venue    0
dtype: int64

<font size='3'> This cluster only has one Italian restaurant which is at the 5 th most common venue. It however has a lot of restaurants</font>


In [209]:
#Cluster 5
df4=queens_merged.loc[queens_merged['Cluster Labels'] == 4, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

In [210]:
df4

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Somerville,Park,Eastern European Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant


In [211]:
#Finding if Italian Restaurant is the 1st most common venue in the cluster
df4.loc[df4['1st Most Common Venue']=='Italian Restaurant']

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [212]:
#Findind number of Italian Restaurants in a cluster

#df4
df4[df4 == 'Italian Restaurant'].count()

Neighborhood              0
1st Most Common Venue     0
2nd Most Common Venue     0
3rd Most Common Venue     0
4th Most Common Venue     0
5th Most Common Venue     0
6th Most Common Venue     0
7th Most Common Venue     0
8th Most Common Venue     0
9th Most Common Venue     0
10th Most Common Venue    0
dtype: int64

<font size='3'>This cluster only has three restaurants but neither of them is Italian.</font>4